Este caderno Jupyter se dedica a criar uma base sintética de benefícios do INSS contendo algumas fraudes. O caso foi baseado em um caso descrito da seguinte forma:

"A fraude foi detectada a partir de uma análise encaminhada pelo INSS. A PF disse também que o servidor investigado reativou temporariamente benefícios cessados por morte, alterando tanto o motivo da interrupção quanto os dados do titular, para receber os valores das aposentadorias na conta dele."
fonte: https://g1.globo.com/pe/pernambuco/noticia/2023/07/04/servidor-do-inss-e-investigado-por-ressuscitar-beneficiarios-mortos-e-desviar-dinheiro-de-aposentadorias.ghtml.


A base sintética possui as seguintes entidades com suas respectivas colunas:

Tabela de servidores:
- id_servidor: Matrícula do servidor concedente (0,1,2,...)
- uf_servidor: Unidade da Federação do servidor concedente (AC, AL, AP, ...)

Tabela de benefícios:
- id_beneficio: Número do Benefício (0,1,2,...)
- uf_beneficiario: Unidade da Federação do beneficiário  (AC, AL, AP, ...)
- id_servidor: Matrícula do servidor concedente
- uf_servidor: Unidade da Federação do servidor concedente (AC, AL, AP, ...)
- uf_coincidente: Indicação se a UF do servidor e do beneficiário é igual (true/false)
- reativado: Indicação se o benefício foi reativado (true/false);
- valor_beneficio: Valor do benefício.
- tempo_analise: Tempo de análise.


In [1]:
# prompt: Gere a tabela de servidores, contendo 500 registros.
# Salve em um dataframe servidores e em um arquivo servidores.csv

import pandas as pd
import numpy as np

# Define a seed for reproducibility
np.random.seed(42)

# Number of server records
n_servers = 500

# Create a list of Brazilian states
ufs = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO']

# Generate server data
data = {
    'id_servidor': range(n_servers),
    'uf_servidor': np.random.choice(ufs, size=n_servers)
}

# Create the DataFrame
servidores = pd.DataFrame(data)

# Save the DataFrame to a CSV file
servidores.to_csv('servidores.csv', index=False)

# Display the first 5 rows of the DataFrame
print(servidores.head())


   id_servidor uf_servidor
0            0          DF
1            1          RN
2            2          PB
3            3          MT
4            4          ES


In [2]:
# prompt: Gere a tabela de benefícios, contendo 100.000 registros.
# Salve em um dataframe beneficios e em um arquivo beneficios.csv
# Use a seguinte distribuição nas seguintes colunas:
# uf_coincidente: verdadeiro em 90% das vezes
# reativado: verdadeiro em 5% das vezes
# valor_beneficio: log-normal (média=log(2000), std=0.1)
# tempo_analise: gamma(alpha=3, scale=1)+5+valor_beneficio/1000+reativado*2
# Considere que 20 servidores são fraudadores. Desses, 3% dos benefícios são fraudados.
# Os benefícios fraudados possuem outras características:
# uf_coincidente: verdadeiro em 99% das vezes
# reativado: sempre verdadeiro
# valor_beneficio: log-normal (média=log(2500), std=0.05)
# tempo_analise: gamma(alpha=2, scale=1)+8
# insira dos dados gerados uma coluna "fraude" indicando se o benefício foi fraudado ou não (true/false)

import pandas as pd
import numpy as np

# Define a seed for reproducibility
np.random.seed(42)

# Number of benefit records
n_beneficios = 100000

# Generate benefit data
data = {
    'id_beneficio': range(n_beneficios),
    'uf_beneficiario': np.random.choice(ufs, size=n_beneficios),
    'id_servidor': np.random.choice(range(n_servers), size=n_beneficios),
    'uf_coincidente': np.random.choice([True, False], size=n_beneficios, p=[0.9, 0.1]),
    'reativado': np.random.choice([True, False], size=n_beneficios, p=[0.05, 0.95]),
    'valor_beneficio': np.random.lognormal(mean=np.log(2000), sigma=0.1, size=n_beneficios),
    'fraude': np.random.choice([True,False], size=n_beneficios, p=[0.00,1.0])
}

# Create the DataFrame
beneficios = pd.DataFrame(data)

# Simulate fraudulent benefits
n_fraudulent_servers = 20
fraudulent_server_ids = np.random.choice(range(n_servers), size=n_fraudulent_servers, replace=False)
fraud_rate = 0.03

for i in range(n_beneficios):
  if beneficios.loc[i,'id_servidor'] in fraudulent_server_ids:
    if np.random.rand() < fraud_rate:
        beneficios.loc[i, 'fraude'] = True
        beneficios.loc[i, 'uf_coincidente'] = np.random.choice([True, False], p=[0.99, 0.01])
        beneficios.loc[i, 'reativado'] = True
        beneficios.loc[i, 'valor_beneficio'] = np.random.lognormal(mean=np.log(2500), sigma=0.05)


# Calculate tempo_analise
beneficios['tempo_analise'] = np.random.gamma(shape=3, scale=1, size=n_beneficios) + 5 + beneficios['valor_beneficio'] / 1000 + beneficios['reativado'] * 2
for i in range(n_beneficios):
  if beneficios.loc[i,'fraude']:
    beneficios.loc[i, 'tempo_analise'] = np.random.gamma(shape=2, scale=1) + 8
# Save the DataFrame to a CSV file
beneficios.to_csv('beneficios.csv', index=False)


In [3]:
# prompt: Faça uma análise descritiva sobre todos os servidores, criando um novo dataframe com as seguintes colunas:
# id_servidor: Matrícula do servidor concedente
# num_beneficios: número de benefícios concedido por cada servidor
# percentual_reativado: percentual de reativações por servidor
# percentual_uf_coincidente: percentual de UFs coincidentes entre servidor e beneficiário
# media_valor: valor médio do benefício por servidor
# media_tempo: tempo médio de análise por servidor
# fraude: indicação se o servidor cometeu alguma fraude (true/false)
# Salve em um dataframe analise_servidores e em um arquivo analise_servidores.csv

import pandas as pd
import numpy as np

# Load the datasets
servidores = pd.read_csv('servidores.csv')
beneficios = pd.read_csv('beneficios.csv')

# Merge dataframes
merged_df = pd.merge(servidores, beneficios, on='id_servidor', how='inner')

# Group data by server
analise_servidores = merged_df.groupby('id_servidor').agg(
    num_beneficios=('id_beneficio', 'count'),
    percentual_reativado=('reativado', lambda x: (x.sum() / len(x)) * 100 if len(x) > 0 else 0),
    percentual_uf_coincidente=('uf_coincidente', lambda x: (x.sum() / len(x)) * 100 if len(x) > 0 else 0),
    media_valor=('valor_beneficio', 'mean'),
    media_tempo=('tempo_analise', 'mean'),
    fraude=('fraude', 'max')
).reset_index()

# Fill NaN values with False for fraude column (assuming no fraud if no benefits)
analise_servidores['fraude'] = analise_servidores['fraude'].fillna(False)

# Rename columns for clarity
analise_servidores = analise_servidores.rename(columns={'id_servidor': 'id_servidor'})

#Save to CSV
analise_servidores.to_csv('analise_servidores.csv', index=False)

print(analise_servidores.head())


   id_servidor  num_beneficios  percentual_reativado  \
0            0             202              3.960396   
1            1             200              4.500000   
2            2             192             10.937500   
3            3             219              2.283105   
4            4             192              6.250000   

   percentual_uf_coincidente  media_valor  media_tempo  fraude  
0                  94.059406  1988.223945    10.192713   False  
1                  85.000000  2023.539196    10.092463   False  
2                  86.458333  2023.542483     9.962347    True  
3                  89.954338  2017.540348     9.990830   False  
4                  90.625000  2014.788905    10.269233   False  
